In [62]:
import pandas as pd
import numpy as np
import os
import glob
import time
import warnings

from functools import reduce

# Additional pandas settings
pd.set_option('max_row', None)
warnings.filterwarnings("ignore")

In [2]:
# Set the working directory
os.chdir('/Users/aakritigupta/Desktop/Hackathon 2021/CSV Files')
os.getcwd()

'/Users/aakritigupta/Desktop/Hackathon 2021/CSV Files'

In [109]:
# Function to read in the data and clean it up 
def readData(csvFile):
  # Read in dataset
  df = pd.read_csv(csvFile)

  # Extract YEAR out of column and store in new column
  df.SERIALNO = df.SERIALNO.map(str)
  df['YEAR'] = df.SERIALNO.str[:4]

  # Convert YEAR to datetime object and filter dataset to only keep 2016 and 2017 data
  df['YEAR'] = pd.to_datetime(df['YEAR']).dt.year
#   df2 = df[(df['YEAR'] == 2017)]
#   df2 = df[(df['YEAR'] == 2018)]
  df2 = df[(df['YEAR'] == 2019)]
  
  return df2

In [110]:
# Function to drop unnecessary columns
def dropCols(df):
  # Create a list of all columns that need to be dropped
  drop_cols = []

  # Add person weight indicators to the list
  for i in df.columns:
    if ('PWGTP' in i) & (len(i) >= 6) or  (i[0]=='F'):
      drop_cols.append(i)

  # Add the specific columns identified after walking through the dataset manually
  drop_cols.extend(['SERIALNO','POBP','RT','DIVISION','SPORDER','PUMA','RELSHIPP','ANC','ANC1P','ANC2P','QTRBIR','RAC2P','RAC3P','OC','RC','ENG','JWRIP','MARHYP','WKWN','YOEP','DECADE','DRIVESP',"JWAP","JWDP",'LANP','NAICSP','MIGPUMA','MIGSP','MSP','NOP','PAOC','POWPUMA','POWSP','SCIENGP','SCIENGRLP','SOCP','VPS','CITWP'])

  # Drop all columns from the dataframe
  df2 = df.drop(columns=drop_cols)

  # Drop duplicate rows
  df3 = df2.drop_duplicates()

  return df3

In [111]:
# Function to impute missing values - Numerical and Categorical features
def missingVals(df):
  # Create a list of numerical columns
  Numerical = ['PWGTP','AGEP','INTP','JWMNP','OIP','PAP','RETP','SEMP','SSIP','SSP','WAGP','WKHP','INDP','PERNP','PINCP','POVPIP']

  # For all missing values in that list, replace with the mean of the column
  for i in Numerical:
    df[i].fillna((df[i].mean()), inplace=True)
  
  # Create a list of categorical columns
  Categorical = []

  # For all missing values in categorical columns, enter category NoInput and convert dtype
  for j in df.columns:
    if j not in Numerical:
      df[j].fillna('NoInput', inplace=True)
      df[j] = df[j].astype('category')
      Categorical.append(j)
  
  return df 

In [112]:
# Function to encode categorical variables
# def oheCat(df, fileNum):
def oheCat(df):
  # Extract the year column from the dataframe
  date = df['YEAR']
  state = df['ST']

  # Drop the year column from the dataframe for encoding purposes
  df2 = df.drop(columns=['YEAR', 'ST'])

  # Create a list of all identified numerical variables
  Numerical = ['PWGTP','AGEP','INTP','JWMNP','OIP','PAP','RETP','SEMP','SSIP','SSP','WAGP','WKHP','INDP','PERNP','PINCP','POVPIP']

  # Create a list of all categorical columns
  cat = []
  for i in df2.columns:
    if (i not in Numerical):
      cat.append(i)
  
  # One hot encode all of the categorical variables
  df3 = pd.get_dummies(df2, prefix=cat)

  # Add the YEAR column back into the dataset
  df3['YEAR'] = date
  df3['ST'] = state

  # Write the final dataset back to the Google Drive folder
  # filePath = "/content/gdrive/MyDrive/Hackathon_2021/data/Processed_files/state_" + str(fileNum) + '.csv'
  # df1.to_csv(filePath)
  
  return df3

In [113]:
# Function to create the percentages in each column
def agg(df):
  # Create the list of numerical features
  Numerical = ['PWGTP','AGEP','INTP','JWMNP','OIP','PAP','RETP','SEMP','SSIP','SSP','WAGP','WKHP','INDP','PERNP','PINCP','POVPIP']
  cat = []

  # Create list of categorical features
  for i in df.columns:
    if (i not in Numerical) & (i != 'YEAR') & (i != 'ST'):
      cat.append(i)
    
    # Calculate mean of entire column for numerical features
    elif (i in Numerical) & (i != 'YEAR'):
      df[i] = df[i].mean()

  # Calculate a proportion of the categorical columns
  for j in cat:
    df[j] = df[j].sum()/len(df)

  # Remove duplicates
  df2 = df.drop_duplicates()
  
  return df2

In [120]:
# Function to iterate through all of the files and run each step of the process
def allFilesProcess():
  lstOfDfs = []
  for i, file in enumerate(os.listdir()):
    if ('.csv' in file) & (i > 40):
      data = readData(file)
      data2 = dropCols(data)
      data3 = missingVals(data2)
      data4 = oheCat(data3)
      data5 = agg(data4)
      
      # Add the df to the list 
      lstOfDfs.append(data5)
      print('Data ' + str(i) + ' has been added to the list')
  
  dfs = [df.reset_index() for df in lstOfDfs]
  dfs_final = pd.concat(dfs, axis=0)
  return dfs_final

In [116]:
%%time
valDf1 = allFilesProcess()
valDf1.shape

Data 0 has been added to the list
Data 1 has been added to the list
Data 2 has been added to the list
Data 3 has been added to the list
Data 4 has been added to the list
Data 5 has been added to the list
Data 6 has been added to the list
Data 7 has been added to the list
Data 8 has been added to the list
Data 9 has been added to the list
Data 10 has been added to the list
Data 12 has been added to the list
Data 13 has been added to the list
Data 14 has been added to the list
Data 15 has been added to the list
Data 16 has been added to the list
Data 17 has been added to the list
Data 18 has been added to the list
Data 19 has been added to the list
Data 20 has been added to the list
CPU times: user 4min 1s, sys: 1min 18s, total: 5min 19s
Wall time: 5min 29s


(20, 874)

In [118]:
%%time
valDf2 = allFilesProcess()
valDf2.shape

Data 21 has been added to the list
Data 22 has been added to the list
Data 23 has been added to the list
Data 24 has been added to the list
Data 25 has been added to the list
Data 26 has been added to the list
Data 27 has been added to the list
Data 28 has been added to the list
Data 29 has been added to the list
Data 30 has been added to the list
Data 31 has been added to the list
Data 32 has been added to the list
Data 33 has been added to the list
Data 34 has been added to the list
Data 35 has been added to the list
Data 36 has been added to the list
Data 37 has been added to the list
Data 38 has been added to the list
Data 39 has been added to the list
Data 40 has been added to the list
CPU times: user 2min 27s, sys: 32.4 s, total: 2min 59s
Wall time: 3min


(20, 875)

In [121]:
%%time
valDf3 = allFilesProcess()
valDf3.shape

Data 41 has been added to the list
Data 42 has been added to the list
Data 43 has been added to the list
Data 44 has been added to the list
Data 45 has been added to the list
Data 46 has been added to the list
Data 47 has been added to the list
Data 48 has been added to the list
Data 49 has been added to the list
Data 50 has been added to the list
Data 51 has been added to the list
Data 52 has been added to the list
CPU times: user 1min 45s, sys: 24.1 s, total: 2min 9s
Wall time: 2min 10s


(12, 872)

In [122]:
# Function to combine all of the files together
def combineFiles(dfa, dfb, dfc):
    dfAll = pd.concat([dfa, dfb, dfc], axis=0)

    # Drop Puerto Rico from the dataset
    dfAll1 = dfAll.loc[(dfAll['ST'] != 72)]

    # Drop duplicates from the dataset
    dfAll1.drop_duplicates(inplace=True)

    return dfAll1

In [123]:
completeDf = combineFiles(valDf1, valDf2, valDf3)
completeDf.head()

,index,PWGTP,AGEP,INTP,JWMNP,OIP,PAP,RETP,SEMP,SSIP,...,REGION_2,NWAV_4.0,OCCP_8510.0,OCCP_8730.0,OCCP_8940.0,RACNUM_6,SFN_3.0,REGION_9,REGION_1,SFN_4.0
0,35956,20.791180,45.034695,2882.260464,26.265063,654.560010,23.013111,6665.239536,1630.566062,255.055472,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,274265,20.668045,42.409024,2760.266599,26.823660,614.216611,20.872907,4388.770110,1920.396327,246.417240,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,31501,18.914936,41.606644,3989.927547,19.900990,936.388469,37.607523,2918.113149,5001.416988,199.830430,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,473477,19.583745,42.743002,2079.826506,24.689704,568.254658,31.152122,4172.732436,1525.682569,315.128169,...,1.0,0.000008,0.000126,0.000110,0.000160,0.000025,NaN,NaN,NaN,NaN
0,119728,19.923219,42.389087,1494.386997,22.861402,732.928268,22.703175,3070.831936,1543.344911,320.637444,...,NaN,NaN,NaN,0.000066,0.000264,NaN,NaN,NaN,NaN,NaN


In [124]:
completeDf.shape

(51, 876)

In [125]:
# Function to combine all of the additional datasets
# def addNewData():
#     # Read in all of the datasets
#     pmh = pd.read_csv('/Users/aakritigupta/Desktop/Hackathon 2021/Poor_ Mental_health_Days_2016.csv')
#     pmh.rename(columns={'STATE': 'State', 'VALUE':'pmh_Value'}, inplace=True)
#     pmh.drop(columns=['RANK'], inplace=True)
    
#     ob = pd.read_csv('/Users/aakritigupta/Desktop/Hackathon 2021/Obesity_2016.csv')
#     ob['Value'] = ob['Value'].str.rstrip('%').astype('float') / 100.0
#     ob.rename(columns={'Value': 'ob_Value'}, inplace=True)
#     ob.drop(columns=['Rank'], inplace=True)
    
#     isl = pd.read_csv('/Users/aakritigupta/Desktop/Hackathon 2021/Insufficient_sleep_2016.csv')
#     isl['Value'] = isl['Value'].str.rstrip('%').astype('float') / 100.0
#     isl.rename(columns={'Value': 'isl_Value'}, inplace=True)
#     isl.drop(columns=['Rank '], inplace=True)
    
#     fmd = pd.read_csv('/Users/aakritigupta/Desktop/Hackathon 2021/Frequent Mental Distress_2016.csv')
#     fmd.rename(columns={'Value': 'fmd_Value'}, inplace=True)
#     fmd.drop(columns=['Rank '], inplace=True)
    
#     air = pd.read_csv('/Users/aakritigupta/Desktop/Hackathon 2021/Air_Pollution_2016.csv')
#     air.rename(columns={'Value': 'air_Value'}, inplace=True)
#     air.drop(columns=['Rank'], inplace=True)

#     # Join the datasets on the State Name
#     lst = [pmh, ob, isl, fmd, air]
#     df_complete = reduce(lambda left, right: pd.merge(left, right, on='State'), lst)
        
#     return df_complete

In [126]:
# newData = addNewData()
# newData.head()

In [127]:
# Function to add in the state initials to the dataset 
def addStInit(df):
    abbr = pd.read_csv('/Users/aakritigupta/Desktop/Hackathon 2021/ST_Abbr_Lkp.csv', header=None)
    abbr2 = abbr.rename(columns={0:'ST', 1:'State', 2:'LocationAbbr'})
    abbr3 = abbr2[(abbr2['ST'] != 72) | (abbr2['ST'] != 11)]

    # Join the abbreviations into the dataset
    df1 = df.merge(abbr3, how='left', on='ST')

    # Add the labels to the dataset - 2017
    # label = pd.read_csv('/Users/aakritigupta/Desktop/Hackathon 2021/cdc_data_17.csv')

    # Add the labels to the dataset - 2018
    # label_df = pd.read_csv('/Users/aakritigupta/Desktop/Hackathon 2021/cdc_data_1819.csv')
    # label_df.drop(columns=['Unnamed: 0'], inplace=True)
    # label18 = label_df.loc[label_df['YearStart'] == 2018]

    # Add the labels to the dataset - 2019
    label_df = pd.read_csv('/Users/aakritigupta/Desktop/Hackathon 2021/cdc_data_1819.csv')
    label_df.drop(columns=['Unnamed: 0'], inplace=True)
    label19 = label_df.loc[label_df['YearStart'] == 2019]

    df2 = df1.merge(label19, how='left', on='LocationAbbr')

    # Drop Puerto Rico from the dataset
    df2.drop((df2[df2['LocationAbbr'] == 'PR'].index) | (df2[df2['LocationAbbr'] == 'DC'].index), inplace=True)

    # Add in the additional data features to the dataset
    # df3 = df2.merge(addDf, how='left', on='State')

    # Drop duplicates from the dataset
    df3 = df2.drop_duplicates()

    return df3

In [128]:
completeDf2 = addStInit(completeDf)
completeDf2.head()

,index_x,PWGTP,AGEP,INTP,JWMNP,OIP,PAP,RETP,SEMP,SSIP,...,RACNUM_6,SFN_3.0,REGION_9,REGION_1,SFN_4.0,State,LocationAbbr,index_y,YearStart,Label
0,35956,20.791180,45.034695,2882.260464,26.265063,654.560010,23.013111,6665.239536,1630.566062,255.055472,...,NaN,NaN,NaN,NaN,NaN,Delaware,DE,27.0,2019.0,0.0
1,274265,20.668045,42.409024,2760.266599,26.823660,614.216611,20.872907,4388.770110,1920.396327,246.417240,...,NaN,NaN,NaN,NaN,NaN,Arizona,AZ,34.0,2019.0,1.0
2,31501,18.914936,41.606644,3989.927547,19.900990,936.388469,37.607523,2918.113149,5001.416988,199.830430,...,NaN,NaN,NaN,NaN,NaN,North Dakota,ND,3.0,2019.0,0.0
3,473477,19.583745,42.743002,2079.826506,24.689704,568.254658,31.152122,4172.732436,1525.682569,315.128169,...,0.000025,NaN,NaN,NaN,NaN,Ohio,OH,38.0,2019.0,1.0
4,119728,19.923219,42.389087,1494.386997,22.861402,732.928268,22.703175,3070.831936,1543.344911,320.637444,...,NaN,NaN,NaN,NaN,NaN,Arkansas,AR,36.0,2019.0,1.0


In [129]:
# Function to impute missing values on aggregated dataset
def missVals(df):
    for i in df.columns:
        missingVals = df[i].isnull().sum()
        if missingVals > 0:
            df[i].fillna(0.0, inplace=True)
    
    return df

In [130]:
valData3 = missVals(completeDf2)
valData3.head()

,index_x,PWGTP,AGEP,INTP,JWMNP,OIP,PAP,RETP,SEMP,SSIP,...,RACNUM_6,SFN_3.0,REGION_9,REGION_1,SFN_4.0,State,LocationAbbr,index_y,YearStart,Label
0,35956,20.791180,45.034695,2882.260464,26.265063,654.560010,23.013111,6665.239536,1630.566062,255.055472,...,0.000000,0.0,0.0,0.0,0.0,Delaware,DE,27.0,2019.0,0.0
1,274265,20.668045,42.409024,2760.266599,26.823660,614.216611,20.872907,4388.770110,1920.396327,246.417240,...,0.000000,0.0,0.0,0.0,0.0,Arizona,AZ,34.0,2019.0,1.0
2,31501,18.914936,41.606644,3989.927547,19.900990,936.388469,37.607523,2918.113149,5001.416988,199.830430,...,0.000000,0.0,0.0,0.0,0.0,North Dakota,ND,3.0,2019.0,0.0
3,473477,19.583745,42.743002,2079.826506,24.689704,568.254658,31.152122,4172.732436,1525.682569,315.128169,...,0.000025,0.0,0.0,0.0,0.0,Ohio,OH,38.0,2019.0,1.0
4,119728,19.923219,42.389087,1494.386997,22.861402,732.928268,22.703175,3070.831936,1543.344911,320.637444,...,0.000000,0.0,0.0,0.0,0.0,Arkansas,AR,36.0,2019.0,1.0


In [131]:
# Function to read in the dataset
def filterDf(df):
    # Read in the dataset
    fs = pd.read_csv('https://raw.githubusercontent.com/aagupta/MentalAid/main/stats_train.csv')
    
    # Drop the unnecessary columns
    fs2 = fs.drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'Label_y'])
    fsLabel = fs['Label_y']
    state = df['State']

    # Create a list of important features
    fsCols = fs2.columns.tolist()

    # Subset the dataframe
    fs3 = df[fsCols]

    # Write the dataset
    # fs3.to_csv('/Users/aakritigupta/Desktop/Hackathon 2021/MentSea/MentSea/Validation_2017.csv')
    fs3.to_csv('/Users/aakritigupta/Desktop/Hackathon 2021/MentSea/MentSea/Validation_2019.csv')

    return (fs3, fsLabel, state)

In [132]:
(valData4, label, state) = filterDf(valData3)
valData4.head()

,JWTRNS_8.0,HISP_2,HISP_4,HISP_21,HISP_24,OCCP_6240.0,OCCP_9830.0,RAC1P_3,RAC1P_7,RAC1P_9,...,OIP,PAP,RETP,SEMP,SSIP,SSP,WAGP,PERNP,PINCP,POVPIP
0,0.000649,0.029075,0.001081,0.000216,0.001729,0.000216,0.001513,0.003351,0.000108,0.026157,...,654.560010,23.013111,6665.239536,1630.566062,255.055472,4798.047151,31282.536561,33324.831504,48191.278366,355.059495
1,0.001691,0.214272,0.002003,0.000341,0.009035,0.000696,0.000852,0.057734,0.001776,0.036737,...,614.216611,20.872907,4388.770110,1920.396327,246.417240,4051.408314,27531.802771,29895.433271,41534.150897,317.284379
2,0.000503,0.022650,0.000378,0.000000,0.002643,0.000378,0.001133,0.045678,0.001007,0.019504,...,936.388469,37.607523,2918.113149,5001.416988,199.830430,3547.653769,28975.946354,34423.107293,45606.884230,343.112497
3,0.000396,0.013955,0.000632,0.000388,0.002039,0.000472,0.000194,0.000986,0.000312,0.023688,...,568.254658,31.152122,4172.732436,1525.682569,315.128169,3681.867809,28113.972425,30071.412929,40488.616693,321.655711
4,0.000660,0.041559,0.000528,0.000264,0.002905,0.000429,0.000363,0.004093,0.001584,0.023536,...,732.928268,22.703175,3070.831936,1543.344911,320.637444,4040.786472,22274.849581,24194.139714,33500.468823,283.596783


## Run code after predictions have been made

In [133]:
# Function to add label back into the dataset to compare
def addLabel(lst, stateLst):
    # Read in the dataset
    # pred = pd.read_csv('/Users/aakritigupta/Desktop/Hackathon 2021/MentSea/MentSea/Predictions_2017.csv')
    # pred = pd.read_csv('/Users/aakritigupta/Desktop/Hackathon 2021/MentSea/MentSea/Predictions_2018.csv')
    pred = pd.read_csv('/Users/aakritigupta/Desktop/Hackathon 2021/MentSea/MentSea/Predictions_2019.csv')

    # Drop unnecessary columns
    pred.drop(columns=['Unnamed: 0'], inplace=True)

    # Join the label to the dataset and the state column
    pred['Label'] = lst

    # Add a condition if values don't match
    pred['Match?'] = np.where(pred['Predictions'] != pred['Label'], 'False', 'True')

    # Add in the states to the dataset
    pred['State'] = state

    # Write df to csv
    # pred.to_csv('/Users/aakritigupta/Desktop/Hackathon 2021/MentSea/MentSea/PredWLabel_2017.csv')
    # pred.to_csv('/Users/aakritigupta/Desktop/Hackathon 2021/MentSea/MentSea/PredWLabel_2018.csv')
    pred.to_csv('/Users/aakritigupta/Desktop/Hackathon 2021/MentSea/MentSea/PredWLabel_2019.csv')

    return pred

In [134]:
%%time
x = addLabel(label, state)

CPU times: user 11.7 ms, sys: 2.47 ms, total: 14.2 ms
Wall time: 13.3 ms


In [135]:
x = pd.read_csv('/Users/aakritigupta/Desktop/Hackathon 2021/MentSea/MentSea/PredWLabel_2018.csv')
for i in x.columns:
    if '_Value' in i:
        print(i)